In [15]:
# this code will read the TFRecords 
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
# this is the set of parameters you want to change during the training to suit your computational need

import FLAGS
reload(FLAGS)
FLAGS.print_all()


Asynchronously process images using multiple threads specified by
the following parameters and the resulting processed images are stored in a
random shuffling queue.
    
FLAGS.batch_size = 32
FLAGS.image_size = 299
FLAGS.num_preprocess_threads = 4
FLAGS.num_readers = 4
FLAGS.input_queue_memory_factor = 16


The fields in a TFRecords example:<br\>
'image/height': _int64_feature(height),<br\>
'image/width': _int64_feature(width),<br\>
'image/colorspace': _bytes_feature(colorspace),<br\>
'image/channels': _int64_feature(channels),<br\>
'image/class/label': _int64_feature(label),<br\>
'image/class/synset': _bytes_feature(synset),<br\>
'image/class/text': _bytes_feature(human),<br\>
'image/object/bbox/xmin': _float_feature(xmin),<br\>
'image/object/bbox/xmax': _float_feature(xmax),<br\>
'image/object/bbox/ymin': _float_feature(ymin),<br\>
'image/object/bbox/ymax': _float_feature(ymax),<br\>
'image/object/bbox/label': _int64_feature([label] * len(xmin)),<br\>
'image/format': _bytes_feature(image_format),<br\>
'image/filename': _bytes_feature(os.path.basename(filename)),<br\>
'image/encoded': _bytes_feature(image_buffer)}))

In [3]:
def decode_jpeg(image_buffer, scope=None):
    """Decode a JPEG string into one 3-D float image Tensor.
    Args:
        image_buffer: scalar string Tensor.
        scope: Optional scope for op_scope.
    Returns:
        3-D float Tensor with values ranging from [0, 1).
    """
    with tf.op_scope([image_buffer], scope, 'decode_jpeg'):
        # Decode the string as an RGB JPEG.
        # Note that the resulting image contains an unknown height and width
        # that is set dynamically by decode_jpeg. In other words, the height
        # and width of image is unknown at compile-time.
        image = tf.image.decode_jpeg(image_buffer, channels=3)

        # After this point, all image pixels reside in [0,1)
        # until the very end, when they're rescaled to (-1, 1).  The various
        # adjust_* ops all require this range for dtype float.
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        return image

In [12]:
def image_preprocessing(image_buffer):
    """Decode and preprocess one image for evaluation or training.
    Args:
        image_buffer: JPEG encoded string Tensor
        train: boolean
        thread_id: integer indicating preprocessing thread
    Returns:
        3-D float Tensor containing an appropriately scaled image
    Raises:
        ValueError: if user does not provide bounding box
    """
     

    image = decode_jpeg(image_buffer)
    height = FLAGS.image_size
    width = FLAGS.image_size
    # Crop the central region of the image with an area containing 87.5% of
    # the original image.
    image = tf.image.central_crop(image, central_fraction=0.875)
    # Resize the image to the original height and width.
    image = tf.expand_dims(image, 0)
    image = tf.image.resize_bilinear(image, [height, width],
                                     align_corners=False)
    image = tf.squeeze(image, [0])

    # Finally, rescale to [-1,1] instead of [0, 1)
    image = tf.sub(image, 0.5)
    image = tf.mul(image, 2.0)
    return image


In [40]:
def parse_example_proto(example_serialized):
    """Parses an Example proto containing a training example of an image.
    The output of the build_image_data.py image preprocessing script is a dataset
    containing serialized Example protocol buffers. Each Example proto contains
    the following fields:
        image/height: 462
        image/width: 581
        image/colorspace: 'RGB'
        image/channels: 3
        image/class/label: 615
        image/class/synset: 'n03623198'
        image/class/text: 'knee pad'
        image/object/bbox/xmin: 0.1
        image/object/bbox/xmax: 0.9
        image/object/bbox/ymin: 0.2
        image/object/bbox/ymax: 0.6
        image/object/bbox/label: 615
        image/format: 'JPEG'
        image/filename: 'ILSVRC2012_val_00041207.JPEG'
        image/encoded: <JPEG encoded string>
    Args:
        example_serialized: scalar Tensor tf.string containing a serialized
        Example protocol buffer.
    Returns:
        image_buffer: Tensor tf.string containing the contents of a JPEG file.
        label: Tensor tf.int32 containing the label.
        bbox: 3-D float Tensor of bounding boxes arranged [1, num_boxes, coords]
        where each coordinate is [0, 1) and the coordinates are arranged as
        [ymin, xmin, ymax, xmax].
        text: Tensor tf.string containing the human-readable label.
    """
    # Dense features in Example proto.
    feature_map = {
        'image/encoded': tf.FixedLenFeature([], dtype=tf.string,
                                          default_value=''),
        'image/class/label': tf.FixedLenFeature([1], dtype=tf.int64,
                                              default_value=-1),
        'image/class/text': tf.FixedLenFeature([], dtype=tf.string,
                                             default_value=''),
    }
    
    features = tf.parse_single_example(example_serialized, feature_map)
    label = tf.cast(features['image/class/label'], dtype=tf.int32)
    

    
    return features['image/encoded'], label, features['image/class/text']

def batch_inputs(dataset, batch_size, shuffle, num_preprocess_threads=None,
                 num_readers=1):
    """Contruct batches of training or evaluation examples from the image dataset.
    Args:
    dataset:  str as either 'train' or 'val'
    batch_size: integer
    shuffle: boolean
    num_preprocess_threads: integer, total number of preprocessing threads
    num_readers: integer, number of parallel readers
    
    Returns:
    images: 4-D float Tensor of a batch of images
    labels: 1-D integer Tensor of [batch_size].
    
    Raises:
    ValueError: if data is not found
    """
    
    with tf.name_scope('batch_processing'):
        # retrieve all the trainning TFRecords in a python list data_file
        tf_record_pattern = os.path.join('/home/sherlock/workspace/ILSVRC2015/TFRecords/','%s-*'%dataset)
        data_files = tf.gfile.Glob(tf_record_pattern)
        data_files = data_files[0:2]
        if data_files is None:
            raise ValueError('No data files found for this dataset')
        # Create filename_queue
        if shuffle:
            filename_queue = tf.train.string_input_producer(data_files,
                                                      shuffle=True,
                                                      capacity=16)
        else:
            print data_files
            filename_queue = tf.train.string_input_producer(data_files,
                                                      shuffle=False,
                                                      capacity=1)
        if num_preprocess_threads is None:
            num_preprocess_threads = FLAGS.num_preprocess_threads 
        if num_preprocess_threads % 4:
            raise ValueError('Please make num_preprocess_threads a multiple '
                               'of 4 (%d % 4 != 0).', num_preprocess_threads)  
                
        if num_readers is None:
            num_readers =  FLAGS.num_readers
        if num_readers < 1:
            raise ValueError('Please make num_readers at least 1')

        # Approximate number of examples per shard. (shard is a partition of the dataset)
        examples_per_shard = 1024
        # Size the random shuffle queue to balance between good global
        # mixing (more examples) and memory use (fewer examples).
        # 1 image uses 299*299*3*4 bytes = 1MB
        # The default input_queue_memory_factor is 16 implying a shuffling queue
        # size: examples_per_shard * 16 * 1MB = 17.6GB
        min_queue_examples = examples_per_shard * FLAGS.input_queue_memory_factor
        if shuffle:
            examples_queue = tf.RandomShuffleQueue(
                capacity=min_queue_examples + 3 * batch_size,
                min_after_dequeue=min_queue_examples,
                dtypes=[tf.string])
        else:
            print 'FIFO'
            examples_queue = tf.FIFOQueue(
                capacity=examples_per_shard + 3 * batch_size,
                dtypes=[tf.string])

        # Create multiple readers to populate the queue of examples.
        if num_readers > 1:
            enqueue_ops = []
            for _ in range(num_readers):
                reader = tf.TFRecordReader()
                _, value = reader.read(filename_queue)
                enqueue_ops.append(examples_queue.enqueue([value]))

            tf.train.queue_runner.add_queue_runner(
                tf.train.queue_runner.QueueRunner(examples_queue, enqueue_ops))
            example_serialized = examples_queue.dequeue()
        else:
            print 'reader %d' %num_readers
            reader = tf.TFRecordReader()
            _, example_serialized = reader.read(filename_queue)

        images_and_labels_and_text = []
        for thread_id in range(num_preprocess_threads):
            # Parse a serialized Example proto to extract the image and metadata.
            image_buffer, label_index, label_text = parse_example_proto(
            example_serialized)
            image = image_preprocessing(image_buffer)
            images_and_labels_and_text.append([image, label_index,label_text])

        images, label_index_batch,label_text_batch = tf.train.batch_join(
            images_and_labels_and_text,
            batch_size=batch_size,
            capacity=2 * num_preprocess_threads * batch_size)

    # Reshape images into these desired dimensions.
        height = FLAGS.image_size
        width = FLAGS.image_size
        depth = 3

        images = tf.cast(images, tf.float32)
        images = tf.reshape(images, shape=[batch_size, height, width, depth])

        # Display the training images in the visualizer.
        tf.image_summary('images', images)

    return images, tf.reshape(label_index_batch, [batch_size]),tf.reshape(label_text_batch, [batch_size])

In [39]:
with tf.Graph().as_default():
    image,label,text = batch_inputs('validation', FLAGS.batch_size, shuffle=False, 
                                    num_preprocess_threads=None,
                                    num_readers=1)
    
    # Image_t is a tensor we define to run the session
    Image_t = tf.add(image,0)
    init_op = tf.group(tf.initialize_all_variables(),
                        tf.initialize_local_variables())
    sess = tf.Session()
    
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    #

    try:
        step = 0
        num=0
        while not coord.should_stop():
            start_time = time.time()

            # Run one step of the model.  The return values are
            # the activations from the `train_op` (which is
            # discarded) and the `loss` op.  To inspect the values
            # of your ops or variables, you may include them in
            # the list passed to sess.run() and the value tensors
            # will be returned in the tuple from the call.
            Image=sess.run([Image_t])

            duration = time.time() - start_time

            # Print an overview fairly often.
            if step % 10 == 0:
                print('Step %d: number of images = %d (%.3f sec)' % (step, num,
                                                     duration))
            step += 1
            num += int(Image_t.get_shape()[0])
            
    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (2, step))
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

        # Wait for threads to finish.
    coord.join(threads)
    sess.close()

['/home/sherlock/workspace/ILSVRC2015/TFRecords/validation-00086-of-00128', '/home/sherlock/workspace/ILSVRC2015/TFRecords/validation-00068-of-00128']
FIFO
reader 1
Step 0: number of images = 0 (0.182 sec)
Step 10: number of images = 320 (0.207 sec)
Step 20: number of images = 640 (0.183 sec)
Step 30: number of images = 960 (0.163 sec)
Step 40: number of images = 1280 (0.164 sec)


KeyboardInterrupt: 

In [25]:
num

array([[[[ 0.92941189,  0.92156875,  0.78039229],
         [ 0.94509816,  0.92156875,  0.77984142],
         [ 0.94509816,  0.92156875,  0.78823543],
         ..., 
         [ 0.91372561,  0.8901962 ,  0.75686288],
         [ 0.90698445,  0.88180208,  0.74901974],
         [ 0.92156875,  0.87450993,  0.74901974]],

        [[ 0.94509816,  0.92156875,  0.78823543],
         [ 0.94509816,  0.92156875,  0.78823543],
         [ 0.94509816,  0.92156875,  0.80281997],
         ..., 
         [ 0.90945411,  0.8859247 ,  0.75259137],
         [ 0.90666938,  0.88313997,  0.74980664],
         [ 0.90666938,  0.88313997,  0.74980664]],

        [[ 0.94352424,  0.91999483,  0.78666151],
         [ 0.94509816,  0.92156875,  0.78969872],
         [ 0.94509816,  0.92156875,  0.8030411 ],
         ..., 
         [ 0.91372561,  0.8901962 ,  0.75686288],
         [ 0.91215169,  0.88862228,  0.75528896],
         [ 0.91299403,  0.88946462,  0.75613129]],

        ..., 
        [[ 0.77018821,  0.58195293,